<a href="https://colab.research.google.com/github/MarcelaMonteiroMontenegroGallo/Python/blob/master/Airflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from airflow.models import DAG
from datetime import datetime

Simple DAG definition

In [ ]:
etl_dag = DAG(
    dag_id = 'etl_pipeline',
    default_args={"start_date":2020-01-08"}
)

In [ ]:
# Define the default_args dictionary
default_args = {
  'owner': 'dsmith',
  'start_date': datetime(2020, 1, 14),
  'retries': 2
}


BashOperator


In [ ]:
from airflow.operators.bash_operator import BashOperator

In [ ]:
BashOperator(
    task_id='bash_exemaplo',
    bash_command ='runcleanup.sh',
    dag=ml_dag
)